#### https://techblog-history-younghunjo1.tistory.com/382

# 1. Activaion Funcion, Loss Function

In [1]:
import numpy as np

# 1. activation func - sigmoid
def sigmoid(x: np.array):
    return 1 / (1 + np.exp(-x))

# 2. activation func - relu
def relu(x: np.array):
    return np.maximum(0, x)

# 3. activation func - softmax 
def softmax(x: np.array):
    if x.ndim == 2:
        x = x.T
        x -= np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T
    
    x -= np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

In [28]:
# 4. loss func - cross-entropy-error
def cross_entropy_error(y: np.array, t: np.array):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    # one-hot vector type
    if t.size == y.size:
        t = t.argmax(axis=1)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [29]:
# 5, 수치 미분 계산 함수
def numerical_gradient(f, x: np.array):
    h = 1e-4
    grads = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        # f(x + h)
        x[idx] = tmp_val + h
        fx1 = f(x)
        
        # f(x - h)
        x[idx] = tmp_val - h
        fx2 = f(x)
        
        grads[idx] = (fx1 - fx2) / (2 * h)
        x[idx] = tmp_val
        it.iternext()
    
    return grads

# 2. 활성화 함수(Relu, Sigmoid) 계층

In [30]:
# 1. Relu Layer
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x:np.array):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
# 2. Sigmoid Layer
class Sigmoid:
    def __init__(self):
        self.y = None
        
    def forward(self, x: np.array):
        y = sigmoid(x)
        self.y = y
        return y
    
    def backward(self, dout):
        dx = dout * self.y * (1 - self.y)
        return dx

# 3. 행렬 곱(Affine) 계층

In [31]:
# 1. Affine Layer
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None
        
    def forward(self, x: np.array):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        
        y = np.matmul(self.x, self.W) + self.b
        return y
    
    def backward(self, dout):
        dx = np.matmul(dout, self.W.T)
        self.dW = np.matmul(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)
        
        return dx

# 4. Softmax, Loss(Cross Entrophy Error) 계층

In [32]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.t = None
        self.y = None
        
    def forward(self, x: np.array, t: np.array):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

# 5. 2-Layer Neural Network

In [33]:
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 2층 신경망의 파라미터 딕셔너리
        self.params = dict()
        self.params['W1'] = np.random.randn(input_size, hidden_size) * weight_init_std
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = np.random.randn(hidden_size, output_size) * weight_init_std
        self.params['b2'] = np.zeros(output_size)
        
        # 2층 신경망의 계층 생성 
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastlayer = SoftmaxWithLoss()
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        loss = self.lastlayer.forward(y, t)
        return loss
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
            
        acc = np.sum(y == t) / float(y.shape[0])
        return acc
    
    def gradient(self, x, t):
        # Forward propagation
        self.loss(x, t)
        
        # Back propagation - 1. Softmax-with-Loss Layer
        dout = 1
        dout = self.lastlayer.backward(dout)
        
        # Back propagation - 2. Rest Layer
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # Back propagation result
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
    
    # 기울기 계산
    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_w, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_w, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
        
        return grads

In [34]:
from datas.mnist import load_mnist

(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

In [35]:
# 2-Layer Neural Network Design

train_size, input_size = X_train.shape
hidden_size = 50
_, output_size = y_train.shape

steps = 1000
batch_size = 100
learning_rate = 0.1

train_loss = []
train_acc = []
test_acc = []

network = TwoLayerNet(input_size=input_size, hidden_size=hidden_size, output_size=output_size)

for i in range(steps):
    batch_mask = np.random.choice(train_size, batch_size)
    X_batch = X_train[batch_mask]
    y_batch = y_train[batch_mask]
    
    # Back propagation
    grads = network.gradient(X_batch, y_batch)
    
    # SGD
    for key in network.params.keys():
        network.params[key] -= learning_rate * grads[key]
        
    # Parameter 갱신 후, get loss
    loss = network.loss(X_batch, y_batch)
    train_loss.append(loss)
    
    if i % 10 == 0:
        tr_acc = network.accuracy(X_batch, y_batch)
        te_acc = network.accuracy(X_test, y_test)
        
        train_acc.append(tr_acc)
        test_acc.append(te_acc)
        
        print("epochs {} - train accuracy:{} - test accuracy:{}".format(i + 1, round(tr_acc, 3), round(te_acc, 3)))

epochs 1 - train accuracy:0.11 - test accuracy:0.092
epochs 11 - train accuracy:0.18 - test accuracy:0.166
epochs 21 - train accuracy:0.26 - test accuracy:0.254
epochs 31 - train accuracy:0.39 - test accuracy:0.302
epochs 41 - train accuracy:0.32 - test accuracy:0.292
epochs 51 - train accuracy:0.41 - test accuracy:0.417
epochs 61 - train accuracy:0.44 - test accuracy:0.424
epochs 71 - train accuracy:0.66 - test accuracy:0.572
epochs 81 - train accuracy:0.68 - test accuracy:0.616
epochs 91 - train accuracy:0.68 - test accuracy:0.697
epochs 101 - train accuracy:0.83 - test accuracy:0.758
epochs 111 - train accuracy:0.78 - test accuracy:0.773
epochs 121 - train accuracy:0.75 - test accuracy:0.783
epochs 131 - train accuracy:0.76 - test accuracy:0.8
epochs 141 - train accuracy:0.85 - test accuracy:0.813
epochs 151 - train accuracy:0.81 - test accuracy:0.824
epochs 161 - train accuracy:0.79 - test accuracy:0.823
epochs 171 - train accuracy:0.84 - test accuracy:0.818
epochs 181 - train accu

## 번외. 검증에 활용되는 수치 미분
- 수치 미분은 상대적으로 오차역전파보다 오래걸리는 특징을 가진다.
- 하지만 직접 수학적 계산을 했기 때문에 해석적 방법을 사용하는 오차역전파 결과를 검증하는데에 사용한다.
    - 기울기 확인 (Gradient Check)
- 두 값의 차이가 0에 가깝다면 오차역전파를 통한 기울기 계산이 잘 되었다고 할 수 있다.

In [36]:
(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=input_size, hidden_size=hidden_size, output_size=10)

# Batch
X_batch = X_train[:3]
y_batch = y_train[:3]

# 수치미분
grad_numerical = network.numerical_gradient(X_batch, y_batch)
grad_propagation = network.gradient(X_batch, y_batch)

for key in grad_numerical.keys():
    diff = np.mean(np.abs(grad_numerical[key] - grad_propagation[key]))
    print('key:{}, diff:{}'.format(key, diff))

key:W1, diff:4.70453984756419e-10
key:b1, diff:2.405896911010009e-09
key:W2, diff:5.9518257291758695e-09
key:b2, diff:1.398746876560275e-07


In [38]:
4.70453984756419e-1

0.470453984756419